# Java 4 — Bazat e të dhënave vektoriale (Vector Databases)

**Objektivat:**
- Të ndërtohet një index vektorial me metadata.
- Të kryhen kërkime semantike dhe filtrime sipas doc_id/doc_type.

**Detyra (Assignment):**
- Ndërto ChromaDB (ose FAISS) me chunks + metadata.
- Demonstro 10 pyetje dhe kthe rezultatet me doc_id/section.

---

# Doctrine & SOP Assistant (Fiktiv) — Google Colab Notebook

**Kujdes / Safety (e detyrueshme):**
- Ky projekt është *arsimor* dhe përdor dokumente **fiktive, të paklasifikuara**.
- Asistenti **nuk** jep këshilla taktike/operacionale.
- Nëse informacioni nuk gjendet në dokumente → **refuzim i sigurt** (“Nuk mbulohet nga dokumentet e disponueshme.”).

---

## 📁 Struktura e të dhënave (rekomandim)

Vendosni PDF-të në:
- `data/raw_pdfs/`

Emrat e rekomanduar (shembull):
- `AAF-TRN-001_SOP_e_Stërvitjes.pdf`
- `AAF-SAFE-010_Rregullat_e_Sigurisë_në_Poligon.pdf`
- `AAF-LOG-004_Siguria_dhe_Kontrolli_i_Mjeteve.pdf`
- `AAF-CMD-007_Zinxhiri_Komandues_dhe_Raportimi_i_Incidenteve.pdf`
- `AAF-GRD-002_Procedurat_e_Sherbimit_te_Rojes.pdf`
- `NATO_Permbledhje_Doktrinore_Edukative.pdf`
- `AAF-IDX-000_Indeksi_Doktrinor_dhe_Procedural.pdf`
- `AAF-ADM-012_Rregulla_Administrative_dhe_Arkivimi.pdf`
- `AAF-RPT-009_Udhezime_te_Pergjithshme_per_Raportim.pdf`


## 0) Setup & Import

In [ ]:
!pip -q install sentence-transformers chromadb pypdf faiss-cpu rapidfuzz

import os, re, json, math
from pathlib import Path


## 1) Ngarko PDF-të (zgjidh një mënyrë)

### Opsioni A — Upload direkt në Colab

In [ ]:
from google.colab import files
uploaded = files.upload()  # zgjidh PDF-të nga kompjuteri
os.makedirs("data/raw_pdfs", exist_ok=True)
for fn in uploaded.keys():
    Path(fn).replace(Path("data/raw_pdfs")/fn)
print("U ngarkuan:", list(uploaded.keys()))


### Opsioni B — Mount Google Drive (rekomanduar për projekte)

In [ ]:
# nga google.colab import drive
# drive.mount('/content/drive')
# Pastaj kopjoni/ndryshoni rrugën sipas vendit ku i keni ruajtur PDF-të, p.sh.:
# !mkdir -p data/raw_pdfs
# !cp -n /content/drive/MyDrive/doctrine_sop_dataset/raw_pdfs/*.pdf data/raw_pdfs/


## 2) Përgatit tekst + chunks + metadata

In [ ]:
from pypdf import PdfReader

def pdf_to_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    parts = []
    for i, page in enumerate(reader.pages):
        txt = page.extract_text() or ""
        parts.append(txt)
    return "\n".join(parts)

pdf_dir = Path("data/raw_pdfs")
pdfs = sorted(pdf_dir.glob("*.pdf"))
print("PDF count:", len(pdfs))
for p in pdfs[:5]:
    print("-", p.name)


In [ ]:
import re

def simple_section_chunk(text: str, max_chars: int = 2500, overlap: int = 300):
    # chunk by headings (very simple heuristic) then by size
    text = re.sub(r'\r\n', '\n', text)
    lines = [ln.strip() for ln in text.split('\n')]
    # treat lines that look like headings: start with digit + '.' or all caps short
    blocks = []
    cur = []
    for ln in lines:
        if re.match(r'^\d+\.', ln) and cur:
            blocks.append("\n".join(cur).strip())
            cur = [ln]
        else:
            if ln:
                cur.append(ln)
    if cur:
        blocks.append("\n".join(cur).strip())

    chunks = []
    for b in blocks:
        if len(b) <= max_chars:
            chunks.append(b)
        else:
            start = 0
            while start < len(b):
                end = min(len(b), start + max_chars)
                chunks.append(b[start:end])
                start = max(0, end - overlap)
    return [c for c in chunks if c.strip()]


In [ ]:
from sentence_transformers import SentenceTransformer

EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL)
print("Loaded:", EMB_MODEL)


In [ ]:
import chromadb
from chromadb.utils import embedding_functions

os.makedirs("data/chroma", exist_ok=True)
client = chromadb.PersistentClient(path="data/chroma")

# Use sentence-transformers embedding function inside Chroma
ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")

col = client.get_or_create_collection(
    name="doctrine_sop",
    embedding_function=ef,
    metadata={"hnsw:space": "cosine"}
)

# (Re)index
# For teaching: delete existing ids (simple reset)
try:
    client.delete_collection("doctrine_sop")
except Exception:
    pass
col = client.get_or_create_collection(
    name="doctrine_sop",
    embedding_function=ef,
    metadata={"hnsw:space": "cosine"}
)

ids, docs, metas = [], [], []
for p in sorted(Path("data/raw_pdfs").glob("*.pdf")):
    doc_id = p.stem
    text = pdf_to_text(str(p))
    chunks = simple_section_chunk(text)
    for j, ch in enumerate(chunks):
        ids.append(f"{doc_id}::chunk{j}")
        docs.append(ch)
        metas.append({"doc_id": doc_id, "chunk": j})

if ids:
    col.add(ids=ids, documents=docs, metadatas=metas)
    print("Indexed chunks:", len(ids))
else:
    print("No PDFs found.")


## 3) Kërkim në Vector DB

In [ ]:
def retrieve(query, k=5, where=None):
    res = col.query(query_texts=[query], n_results=k, where=where)
    out = []
    for i in range(len(res["ids"][0])):
        out.append({
            "id": res["ids"][0][i],
            "doc_id": res["metadatas"][0][i].get("doc_id"),
            "score": res["distances"][0][i],
            "snippet": (res["documents"][0][i][:240]).replace("\n"," ")
        })
    return out

q = "Brenda sa orëve duhet raportuar incidenti?"
for r in retrieve(q, k=5):
    print(r)


## 4) Dorëzim

- Jep 10 pyetje dhe rezultatet top-3.
- Shto të paktën 1 filtër `where={'doc_id': ...}` dhe shpjego pse.